In [ ]:
import pandas as pd

## NOTE THAT NONE OF THESE INDIVIDUATED CAMPAIGN DATAFRAME FOLDERS ARE LIKELY TO WORK! All sensitive data has been stripped.  as per the readme, you can start from the completed campaign data and secured df csv's, or plug your own mailchimp data in here!

## Import CSV's as pandas Dataframes for each individual mailchimp metric in the reports

In [ ]:
dfbounce =  pd.read_csv("members__Copy_01__bounces_May_16_2019.csv")
dfclicks =  pd.read_csv("members__Copy_01__click_activity_May_16_2019.csv")
dfnoopen = pd.read_csv("members__Copy_01__not_opened_May_16_2019.csv")
dfopen = pd.read_csv("members__Copy_01__opened_May_16_2019.csv")
dfsent = pd.read_csv("members__Copy_01__sent_May_16_2019.csv")
dfunsubscribe = pd.read_csv("members__Copy_01__unsub-reasons_May_16_2019.csv")


In [ ]:
# Sum campaign data
df_campaign_sum = pd.read_csv("campaign_sum_data.csv")

In [ ]:
df_campaign_sum

###### Select the row with our specific campaigns sum data 

In [ ]:
campaign_data = df_campaign_sum.iloc[1, 5:10].to_frame()

In [ ]:
campaign_data = campaign_data.T.head()

In [ ]:
campaign_data.reset_index(inplace=True)

In [ ]:
campaign_data['campaign_id'] = campaign_data['index']

In [ ]:
campaign_data = campaign_data.iloc[:, 1:]

In [ ]:
campaign_data

In [ ]:
dfbounce.shape

In [ ]:
dfclicks.shape

In [ ]:
dfnoopen.shape

In [ ]:
dfopen.shape

In [ ]:
dfsent.shape

In [ ]:
dfunsubscribe.shape

# How to anonymize sensitive data in a mailchimp report, using a pandas dataframe

## Generate a blank list, where you can use the extend method with the tolist method to feed list versions of these email addresses from all the metrics in these dataframe.

In [ ]:
full_email_list = [] #Thanks for help with this Thomas Ludlow!

In [ ]:
full_email_list.extend(dfbounce['Email Address'].tolist())

In [ ]:
full_email_list.extend(dfclicks['Email Address'].tolist())

In [ ]:
full_email_list.extend(dfnoopen['Email Address'].tolist())

In [ ]:
full_email_list.extend((dfopen['Email Address'].tolist()))

In [ ]:
full_email_list.extend(dfsent['Email Address'].tolist())  

In [ ]:
full_email_list.extend(dfunsubscribe['Email Address'].tolist())

In [ ]:
len(full_email_list)

In [ ]:
len(set(full_email_list))

### Save as a set, to remove duplicates.  Now you have a concise set of emails

In [ ]:
the_email_list1 = set(full_email_list)

In [ ]:
the_email_list1 = pd.DataFrame(the_email_list1)

In [ ]:
the_email_list1.head()

In [ ]:
the_email_list1["Email Address"] = the_email_list1[0]

In [ ]:
the_email_list1.pop(0).head()

In [ ]:
the_email_list1.shape

In [ ]:
the_email_list1.to_csv("email_list1.csv")

### Use a numpy random shuffling method such as below, to randomly shuffle your emails. 

In [ ]:
import numpy as np

In [ ]:
shuffled_email_list = the_email_list1.reindex(np.random.permutation(the_email_list1.index))

In [ ]:
shuffled_email_list.head()

### Reset your index, so this new shuffled list now has an entirely new one.  Then delete the old 'index' column that contains your original index. 


In [ ]:
shuffled_email_list.reset_index(inplace = True)


In [ ]:
shuffled_email_list.drop(columns=['index'],inplace = True)
shuffled_email_list.head()

In [ ]:
shuffled_email_list.head()

### Set this new index as email_index - you will use this instead of sensitive data to anonymize your audience, once you pd.merge all lists to this index.

In [ ]:
# Or do this process to the full list.  

In [ ]:
shuffled_email_list['email_id'] = shuffled_email_list.index

In [ ]:
shuffled_email_list.head()

#### Save this outside of your repo or other publicly accessible directory.  This should be stored separately from any public git repos or anywhere else publicly accessible, and be used for your purposes only, to link this information back to your audience.

In [ ]:
shuffled_email_list.to_csv("./../../sensitive_info/email_secret_keys.csv")

### Opens and no opens are a simple merger, where you impute the correct value to your no opens dataframe, 0, below.  They are then structured pretty much identically.

In [ ]:
dfnoopen['Opens'] = 0

In [ ]:
dfnoopen

In [ ]:
df_opens = dfopen.merge(dfnoopen, how = "outer")

In [ ]:
df_opens.head()

In [ ]:
df_opens.shape

In [ ]:
df_opens_sent = df_opens.merge(dfsent, how = "outer")

In [ ]:
df_opens_sent.head()

#### the bounce and dataframe set are similar.  Just set the "Bounce type" in sent to "Sent" before merging.

In [ ]:
dfbounce.head()

In [ ]:
dfsent['Bounce Type'] = "sent"

In [ ]:
dfsent.head()

In [ ]:
df_bounce_or_sent = dfbounce.merge(dfsent, how='outer')

In [ ]:
df_bounce_or_sent.head()

In [ ]:
#Our figure here, 1010, shows us we have a complete dataframe
df_bounce_or_sent.shape

#### This is the resultant dataframe we want, for each campaign.

In [ ]:
df_opens_sents = df_opens_sent.merge(df_bounce_or_sent, how='outer')

In [ ]:
df_opens_sents

In [ ]:
df_opens_sents.shape

In [ ]:
df_opens_sents.isnull().sum()

#### Unsubscribes, along with clicks, are more concerned with the content of the email.  So we will also set these aside for now.

In [ ]:
dfunsubscribe['unsubscibe reason'] = dfunsubscribe['reason']

In [ ]:
dfunsubscribe['unsubscribe description'] = dfunsubscribe['description']

In [ ]:
dfunsubscribe.head()

####  Assign email ID, using the secret key generated with all emails, from Dataframe_Cleaning123456

In [ ]:
secret_key = pd.read_csv("./../../sensitive_info/email_secret_keys123456.csv")

In [ ]:
df_opens_sents['campaign_id'] = 1

In [ ]:
df_opens_sents_complete = df_opens_sents.merge(campaign_data, how='right')

In [ ]:
df_opens_sents_complete.head()

#### Assign email ID

In [ ]:
df_opens_sents_completed = df_opens_sents_complete.merge(shuffled_email_list, how = 'right')

In [ ]:
df_opens_sents_completed.head()

#### Remove identifying info 

In [ ]:
#The first member rating's are an input error on my part that needs to be cleaned.
df1secure = df_opens_sents_completed.iloc[:, 9:]

In [ ]:
df1secure.isnull().sum()

#### fill remaining nulls for Opens with 0

In [ ]:
df1secure['Opens'].fillna(0, inplace=True)

In [ ]:
df1secure.isnull().sum()

In [ ]:
df1secure.to_csv("df1secure.csv")